#### Read Data

In [2]:
import graphlab
import numpy as np

sales = graphlab.SFrame('Data1/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1503819469.log


This non-commercial license of GraphLab Create for academic use is assigned to B140007@e.ntu.edu.sg and will expire on August 27, 2018.


#### Convert to numpy

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe["constant"] = 1
    features = ['constant'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return (feature_matrix, output_array)
    

In [5]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


#### Predicting output given regression weights

In [6]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print predicted_value

1181.0


In [7]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions


In [8]:
test_predictions = predict_output(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


#### Compute derivative

In [10]:
def feature_derivative(errors, feature):
    derivative = 2*np.dot(errors, feature)
    
    return derivative



In [11]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850022.0
-23345850022.0


#### Gradient Descent

In [12]:
from math import sqrt

def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    
    while not converged:
        predictions = predict_output(feature_matrix, weights)
        
        errors = predictions - output
        
        gradient_sum_squares = 0
        
        for i in range(len(weights)):
            derivative = feature_derivative(errors, feature_matrix[:,i])
            gradient_sum_squares = gradient_sum_squares + derivative**2
            weights[i] = weights[i] - step_size*derivative
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return weights

#### Run the Gradient Descent

In [13]:
train_data,test_data = sales.random_split(.8,seed=0)

# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

model_weight = regression_gradient_descent(simple_feature_matrix,output,initial_weights,step_size,tolerance)
model_weight

array([-46999.88716555,    281.91211912])

In [21]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)
test_output

array([ 310000.,  650000.,  233000., ...,  610685.,  400000.,  402101.])

#### Predictions

In [22]:
predictions = predict_output(test_simple_feature_matrix, model_weight)
predictions

array([ 356134.44317093,  784640.86422788,  435069.83652353, ...,
        663418.65300782,  604217.10799338,  240550.4743332 ])

In [23]:
np.sum((predictions - test_output)**2)

275400047593155.94

#### Run the multiple regression

In [24]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [25]:
model_weight_2 = regression_gradient_descent(feature_matrix,output,initial_weights,step_size,tolerance)

In [ ]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
test_output